# MP2

## Some useful resources:
 - [original paper](https://journals.aps.org/pr/abstract/10.1103/PhysRev.46.618)
 - Levine Chapter 16
 - [psi4numpy tutorial](https://github.com/psi4/psi4numpy/blob/master/Tutorials/05_Moller-Plesset/5a_conventional-mp2.ipynb)
 - [Crawdad programming notes](http://sirius.chem.vt.edu/wiki/doku.php?id=crawdad:programming:project4)

# MP2 algorithm
1. The starting point will be the Hartree-Fock wavefunction

## Imports

In [1]:
import numpy as np
import scipy.linalg as spla
import psi4
import matplotlib.pyplot as plt
import time
%matplotlib notebook

## Specify the molecule

In [2]:
# start timer
start_time = time.time()
# define molecule
mol = psi4.geometry("""
O 0.0000000 0.0000000 0.0000000
H 0.7569685 0.0000000 -0.5858752
H -0.7569685 0.0000000 -0.5858752
symmetry c1
""")
psi4.set_options({'basis': 'sto-3g'})
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('BASIS'))
mints = psi4.core.MintsHelper(wfn.basisset())
# get number of electrons
num_elec_alpha =  wfn.nalpha()
num_elec_beta =  wfn.nbeta()
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc =  mol.nuclear_repulsion_energy()

# STEP 2 : Calculate molecular integrals 

Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H = T + V $$

Two electron integrals

$$ (\mu\nu|\lambda\sigma) = \int dr_1 dr_2 \phi^*_{\mu}(r_1) \phi_{\nu}(r_1) r_{12}^{-1} \phi_{\lambda}(r_2) \phi_{\sigma}(r_2) $$


In [3]:
# calculate overlap integrals
S = np.asarray(mints.ao_overlap())
# calculate kinetic energy integrals
T = np.asarray(mints.ao_kinetic())
# calculate nuclear attraction integrals
V = np.asarray(mints.ao_potential())
# form core Hamiltonian
H = T + V
# calculate two electron integrals
eri = np.asarray(mints.ao_eri())
# get number of atomic orbitals
num_ao = np.shape(S)[0]



print(np.shape(eri))

(7, 7, 7, 7)


In [4]:
# set inital density matrix to zero
D = np.zeros((num_ao,num_ao))

# 2 helper functions for printing during SCF
def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("Iter", "Time(s)", "RMSC DM", "delta E", "E_scf_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("****", "*******", "*******", "*******", "******")))
def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_scf_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num, iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_scf_elec)))

# set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# loop variables
iteration_num = 0
E_scf_total = 0
E_scf_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False
print_start_iterations()
while (not converged and not exceeded_iterations):
    # store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_scf_elec
    D_last = np.copy(D)
    # form G matrix
    G = np.zeros((num_ao,num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G[i,j] += D[k,l] * ((2.0*(eri[i,j,k,l])) - (eri[i,k,j,l]))
    # build fock matrix
    F  = H + G
    # solve the generalized eigenvalue problem
    E_orbitals, C = spla.eigh(F,S)
    # compute new density matrix
    D = np.zeros((num_ao,num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D[i,j] +=  C[i,k] * C[j,k]
    # calculate electronic energy
    E_scf_elec = np.sum(np.multiply(D , (H +  F)))
    # calculate energy change of iteration
    iteration_E_diff = np.abs(E_scf_elec - E_elec_last)
    # rms change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D - D_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_scf_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM): 
        converged = True
    if(iteration_num == iteration_max):
        exceeded_iterations = True

# calculate total energy
E_scf_total = E_scf_elec + E_nuc
print("{:^79}".format("Total HF energy : {:>11f}".format(E_scf_total)))

           Iter      Time(s)      RMSC DM      delta E   E_scf_elec            
           ****      *******      *******      *******       ******            
              1     0.102164  2.69561E+00  1.27367E+02  -127.366748            
              2     0.010279  1.84626E+00  4.69671E+01   -80.399634            
              3     0.007715  1.84892E-01  4.07021E+00   -84.469843            
              4     0.005661  3.65179E-02  3.36584E-01   -84.133260            
              5     0.008053  1.41819E-02  2.77766E-02   -84.161036            
              6     0.008243  5.65413E-03  2.81184E-03   -84.158224            
              7     0.008635  2.37192E-03  1.60719E-04   -84.158064            
              8     0.010071  1.00945E-03  1.58481E-04   -84.157905            
              9     0.009296  4.33408E-04  6.31723E-05   -84.157842            
             10     0.008787  1.86875E-04  2.79308E-05   -84.157814            
             11     0.011495  8.07615E-0

# Perform MP2 calculation

## Convert the two-electron integrals from AO basis to the MO basis

$$(pq|rs) = \sum_\mu \sum_\nu \sum_\lambda \sum_\sigma C_\mu^p C_\nu^q
(\mu \nu|\lambda \sigma) C_\lambda^r C_\sigma^s.$$

This is implemented in the cell block below. There are a few ways to implement this, below is by far the worst. The algorithm coded below is the naive approach known as the Noddy algorithm. This algorithm scales as $N^8$, although MP2 is formally known to scale as $N^5$; however. The Noddy algorithm is a great starting point.

In [28]:
noddy_start = time.time()
eri_mo = np.zeros((num_ao, num_ao, num_ao, num_ao))
for p in range(num_ao):
    for q in range(num_ao):
        for r in range(num_ao):
            for s in range(num_ao):
                for mu in range(num_ao):
                    for nu in range(num_ao):
                        for lmda in range(num_ao):
                            for sigma in range(num_ao):
                                eri_mo[p, q, r, s] += C[mu, p]*C[nu, q]*C[lmda,r]*C[sigma, s]*eri[mu, nu, lmda, sigma]
                    
noddy_end = time.time()
noddy_time = noddy_end - noddy_start
print(noddy_time)

17.515717029571533


### Compute the MP2 Energy
Now we can calculate the MP2 estimation of the correlation energy. 
$$E_{\mathrm{corr(MP2)}}\ =\ \frac{( ia \mid jb ) [ 2 (ia \mid jb ) - ( ib \mid ja )]}{\epsilon_i + \epsilon_j + \epsilon_a - \epsilon_b}$$

In [6]:
E_corr_mp2 = 0
for i in range(num_elec_alpha):
    for j in range(num_elec_alpha):
        for a in range(num_elec_alpha, num_ao):
            for b in range(num_elec_alpha, num_ao):
                temp = eri_mo[i, a, j, b] * \
                    (2*eri_mo[i, a, j, b] - eri_mo[i, b, j, a])
                temp /= (E_orbitals[i] + E_orbitals[j] - E_orbitals[a] - E_orbitals[b])
                E_corr_mp2 += temp
                
print("{:^79}".format("Total MP2 correlation energy : {:>11f}".format(E_corr_mp2)))

                  Total MP2 correlation energy :   -0.035493                   


The correlation energy is very small compared to the total energy, which is generally the case. However, this correlation energy can be very important to describing properties such as dispersion. 

## A comparison with Psi4

In [8]:
# Get the SCF wavefunction & energies# Get t 
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)
mp2_e = psi4.energy('mp2')
print(mp2_e)

E_diff = (mp2_e - (E_scf_total + E_corr_mp2)) 
print(E_diff)

-74.99850828492245
-8.603406115526013e-05


### An implementation of the smart algorithm

In [24]:
# nbasis, norb = C.shape
import time

smart_start = time.time()
sump = np.zeros((num_ao,num_ao,num_ao,num_ao))
for mu in range(num_ao):
    sumq = np.zeros((num_ao,num_ao,num_ao))
    for nu in range(num_ao):
        sumr = np.zeros((num_ao,num_ao))
        for lmda in range(num_ao):
            sums = np.zeros((num_ao))
            for sigma in range(num_ao):
                for s in range(num_ao):
                    sums[s] += C[sigma,s]*eri[mu,nu,lmda,sigma]
            for r in range(num_ao):
                for s in range(num_ao):
                    sumr[r,s] += C[lmda,r]*sums[s]
        for q in range(num_ao):
            for r in range(num_ao):
                for s in range(num_ao):
                    sumq[q,r,s] += C[nu,q]*sumr[r,s]
    for p in range(num_ao):
        for q in range(num_ao):
            for r in range(num_ao):
                for s in range(num_ao):
                    sump[p,q,r,s] += C[mu,p]*sumq[q,r,s]
eri_mo = sump

smart_end = time.time()
smart_time = smart_end -smart_start
print(smart_time)

0.10822081565856934


In [21]:
E_corr_mp2 = 0
for i in range(num_elec_alpha):
    for j in range(num_elec_alpha):
        for a in range(num_elec_alpha, num_ao):
            for b in range(num_elec_alpha, num_ao):
                temp = eri_mo[i, a, j, b] * \
                    (2*eri_mo[i, a, j, b] - eri_mo[i, b, j, a])
                temp /= (E_orbitals[i] + E_orbitals[j] - E_orbitals[a] - E_orbitals[b])
                E_corr_mp2 += temp
                
print("{:^79}".format("Total MP2 correlation energy : {:>11f}".format(E_corr_mp2)))

                  Total MP2 correlation energy :   -0.035493                   


# algorithm time comparison

In [26]:
ns_time = noddy_time/smart_time
print(ns_time)

151.40355488190417
